In [0]:
from cni_connectors import adls_gen1_connector as adls_conn
var_adls_uri = adls_conn.adls_gen1_connect(spark, dbutils, scope="adls_gen2", dynamic_overwrite="dynamic")

In [0]:
import json
import re
import pyspark.sql.functions as f
from pyspark.sql import DataFrame
from pyspark.sql.window import Window
import pyspark.sql.functions as f
from trs_control_field import trs_control_field as tcf

In [0]:
table = json.loads(re.sub("\"", "\"", dbutils.widgets.get("tables")))
dls = json.loads(re.sub("\"", "\"", dbutils.widgets.get("dls")))
adf = json.loads(re.sub("\"", "\"", dbutils.widgets.get("adf")))

In [0]:
table = {"copy_sqldw":"false",
  "source_pnadc":"ibge/pnadc/",
  "trs_deflatores_path":"oni/ibge/deflatores/",
  "params_to_save_0":"ocupados/",
  "params_to_save_1":"populacao/",
  "path_destination":"oni/mulheres_no_mercado_de_trabalho/pnad_população_ocupados",
  "destination":"/mulheres_no_mercado_de_trabalho/pnad_população_ocupados",
  "databricks":{"notebook":"/biz/oni/mulheres_no_mercado_de_trabalho/trs_biz_pnad_trimestral"}}

adf = {"adf_factory_name":"cnibigdatafactory","adf_pipeline_name":"trs_biz_rfb_cno","adf_pipeline_run_id":"c158e9dd-98df-4f7b-a64d-d13c76669868","adf_trigger_id":"67c514a7245449b984eb4aadd55bfbff","adf_trigger_name":"Sandbox","adf_trigger_time":"2023-08-22T21:22:42.5769844Z","adf_trigger_type":"Manual"}
 
dls = {"folders":{"landing":"/tmp/dev/lnd","error":"/tmp/dev/err","archive":"/tmp/dev/ach","staging":"/tmp/dev/stg","log":"/tmp/dev/log","raw":"/tmp/dev/raw","trusted":"/tmp/dev/trs","business":"/tmp/dev/biz","prm":"/tmp/dev/prm","historico":"/tmp/dev/hst"},"path_prefix":"tmp","uld":{"folders":{"landing":"/tmp/dev/uld","error":"/tmp/dev/err","staging":"/tmp/dev/stg","log":"/tmp/dev/log","raw":"/tmp/dev/raw","archive":"/tmp/dev/ach"},"systems":{"raw":"usr"},"path_prefix":"/tmp/dev/"},"systems":{"raw":"usr"}}

#cadastro_estbl_f = "{adl_path}{trs}/{file_folder}/".format(adl_path=var_adls_uri, trs=dls["folders"]["trusted"], file_folder=table["cadastro_estbl_f"])
#cadastro_estbl = spark.read.format("parquet").option("header","true").option("encoding", "ISO-8859-1").load(cadastro_estbl_f, mode="FAILFAST")

In [0]:
source_pnadc = "{adl_path}{trusted}/{file_folder}/".format(adl_path=var_adls_uri, trusted=dls["folders"]["trusted"], file_folder=table["source_pnadc"])
trs_deflatores_path = "{adl_path}{trusted}/{file_folder}/".format(adl_path=var_adls_uri, trusted=dls["folders"]["trusted"], file_folder=table["trs_deflatores_path"])

In [0]:
df_pnadc = spark.read.parquet(source_pnadc)

In [0]:
deflatores=(spark.read.parquet(trs_deflatores_path))

In [0]:
deflatores_columns = ['Ano','trim','UF','Habitual','Efetivo']
df_deflatores=(deflatores
           .select(*deflatores_columns))
df_deflatores=(deflatores
               .withColumnRenamed('Ano','ANO')
               .withColumn('TRIMESTRE',f.when((f.col('trim')==f.lit('01-02-03')), f.lit(1)).otherwise(
               f.when((f.col('trim')==f.lit('04-05-06')), f.lit(2)).otherwise(
               f.when((f.col('trim')==f.lit('07-08-09')), f.lit(3)).otherwise(
               f.when((f.col('trim')==f.lit('10-11-12')), f.lit(4))))))
              .where(f.col('trim').isNotNull())
           .drop('trim'))

In [0]:
def select_columns_pnadc(df: DataFrame)-> DataFrame:
  pnadc_columns = ["ANO","TRIMESTRE","UF","UPA", "V1008", "V1022", "V1028","V2007","V2009","V3002","V4040","VD3004","VD3005","VD4001", 
                   "VD4002","VD4003","VD4004","VD4004A","VD4005","VD4009","VD4012","VD4017","VD4018","V4019","V4076","V40761","V40762","V40763","V2010","V4013","V4039"]
  return df.select(*pnadc_columns)

In [0]:
df_pnadc = select_columns_pnadc(df_pnadc)

In [0]:
def formating_and_filtering_variables(df: DataFrame) -> DataFrame:
  return df.join(df_deflatores,['ANO','TRIMESTRE','UF']).withColumnRenamed("TRIMESTRE","nm_trimestre")\
          .withColumnRenamed("ANO","nm_ano")\
          .withColumnRenamed("UF","cd_uf")\
          .withColumnRenamed("V4013","cd_cnae_dom")\
          .withColumnRenamed("V2007","cd_sexo")\
          .withColumn('ds_sexo',f.when(f.col("cd_sexo")==f.lit(1),"Masculino")\
                                  .otherwise(f.when(f.col("cd_sexo")==f.lit(2),"Feminino")))

In [0]:
df_pnadc = formating_and_filtering_variables(df_pnadc)

In [0]:
def calculate_employed(df: DataFrame) -> DataFrame:
  return (df_pnadc.withColumn('cd_cnae_dom', f.lpad('cd_cnae_dom',5,'0'))
          .withColumn('cd_cnae_dom_div', f.substring('cd_cnae_dom',1,2))
          .withColumn("vl_ocupados",f.when(f.col("VD4002") == f.lit(1),f.col("V1028")).otherwise(f.lit(0)))\
                 .withColumn("vl_protecao_social",f.when((f.col("VD4002")==f.lit(1)) & \
                                                                ((f.col("VD4009").isin([2, 4,6,10])) | 
                                                                 ((f.col("VD4009").isin([8,9])) & \
                                                                (f.col('VD4012')==f.lit(2)))),f.col('V1028')).otherwise(f.lit(0)))
                 .withColumn('vl_horas_trabalhadas', f.when(f.col('VD4002')==f.lit(1), f.col('V4039')*f.lit(4)*f.col('V1028')).otherwise(f.lit(0))))


In [0]:
df_pnadc = calculate_employed(df_pnadc)

In [0]:
def calculate_population(df: DataFrame) -> DataFrame:
  return (df_pnadc.withColumn("vl_desocupados",f.when(f.col("VD4002") == f.lit(2),f.col("V1028")).otherwise(f.lit(0)))
          .withColumn('vl_populacao_14anosmais', f.when(f.col('V2009') >= f.lit(14), f.col('V1028')).otherwise(f.lit(0)))
          .withColumn("vl_subocupados",f.when((f.col("VD4004A") == f.lit(1)) |(f.col("VD4004") == f.lit(1)),f.col("V1028")).otherwise(f.lit(0)))
          .withColumn("vl_subutilizados",f.when((f.col("VD4004A") == f.lit(1)) |(f.col("VD4004") == f.lit(1)) #subocupados por insuficiências de horas
                                                 | (f.col('VD4003') == f.lit(1)) | #pessoas fora da força de trabalho e na força de trabalho potencial
                                                 (f.col('VD4002')==f.lit(2)) #pessoas desocupadas
                                                 ,f.col("V1028")).otherwise(f.lit(0)))
          .withColumn("vl_forca_de_trabalho",f.when(f.col("VD4001") == f.lit(1),f.col("V1028")).otherwise(f.lit(0)))
          .withColumn("vl_forca_de_trabalho_ampliada",f.when((f.col("VD4001") == f.lit(1)) | (f.col('VD4003') == f.lit(1)),f.col("V1028")).otherwise(f.lit(0))))  


In [0]:
df_pnadc = calculate_population(df_pnadc)

In [0]:
def calculate_salary_and_population(df: DataFrame) -> DataFrame:
  return df_pnadc.withColumn("vl_massa_salarial",f.when(f.col("VD4002") == f.lit(1),(f.col("V1028")*f.col("VD4017")*f.col('Efetivo'))).otherwise(f.lit(0)))


In [0]:
df_pnadc = calculate_salary_and_population(df_pnadc)

In [0]:
def calculate_value_learning(df: DataFrame)-> DataFrame:
  return (df.withColumn("vl_anos_estudos",f.when((f.col("V2009") >=f.lit(5)),(f.col("VD3005")* f.col("V1028")).cast("integer"))
                      .otherwise(f.lit(0)))
           .withColumn("ds_grau_instrucao_agregado", f.when(f.col('VD3004').isin(['1','2','3','4']),f.lit('Até o fundamental completo')).otherwise(
             f.when(f.col('VD3004').isin(['5','6']), f.lit('Médio completo')).otherwise(
               f.when(f.col('VD3004')==f.lit(7), f.lit('Superior completo')).otherwise(f.lit('Ignorado')))))
           .withColumn("cd_grau_instrucao_agregado", f.when(f.col('VD3004').isin(['1','2','3','4']),f.lit(1)).otherwise(
             f.when(f.col('VD3004').isin(['5','6']), f.lit(2)).otherwise(
               f.when(f.col('VD3004')==f.lit(7), f.lit(3)).otherwise(f.lit(-1))))))

In [0]:
df_pnadc = calculate_value_learning(df_pnadc)

In [0]:
def create_age_group(df: DataFrame)-> DataFrame:
  return (df.withColumn("cd_faixa_etaria",f.when(((f.col("V2009")<=f.lit(20))),f.lit(1))\
                                        .otherwise(f.when(((f.col("V2009")>f.lit(20)) & (f.col("V2009")<=f.lit(30))),f.lit(2))\
                                        .otherwise(f.when(((f.col("V2009")>f.lit(30)) & (f.col("V2009")<=f.lit(40))),f.lit(3))\
                                        .otherwise(f.when(((f.col("V2009")>f.lit(40)) & (f.col("V2009")<=f.lit(50))),f.lit(4))\
                                        .otherwise(f.when(((f.col("V2009")>f.lit(50)) & (f.col("V2009")<=f.lit(60))),f.lit(5))\
                                        .otherwise(f.when(((f.col("V2009")>f.lit(60)) & (f.col("V2009")<=f.lit(70))),f.lit(6))\
                                        .otherwise(f.when(((f.col("V2009")>f.lit(70)) & (f.col("V2009")<=f.lit(80))),f.lit(7))\
                                        .otherwise(f.when(((f.col("V2009")>f.lit(80))),f.lit(8)).otherwise(f.lit(9))))))))\
                                        ))\
                       .withColumn("ds_faixa_etaria",f.when(((f.col("V2009")<=f.lit(20))),f.lit('Até 20 anos'))\
                                        .otherwise(f.when(((f.col("V2009")>f.lit(20)) & (f.col("V2009")<=f.lit(30))),f.lit('21 a 30 anos'))\
                                        .otherwise(f.when(((f.col("V2009")>f.lit(30)) & (f.col("V2009")<=f.lit(40))),f.lit('31 a 40 anos'))\
                                        .otherwise(f.when(((f.col("V2009")>f.lit(40)) & (f.col("V2009")<=f.lit(50))),f.lit('41 a 50 anos'))\
                                        .otherwise(f.when(((f.col("V2009")>f.lit(50)) & (f.col("V2009")<=f.lit(60))),f.lit('51 a 60 anos'))\
                                        .otherwise(f.when(((f.col("V2009")>f.lit(60)) & (f.col("V2009")<=f.lit(70))),f.lit('61 a 70 anos'))\
                                        .otherwise(f.when(((f.col("V2009")>f.lit(70)) & (f.col("V2009")<=f.lit(80))),f.lit('71 a 80 anos'))\
                                        .otherwise(f.when(((f.col("V2009")>f.lit(80))),f.lit('Maiores de 80 anos')).otherwise('Não informado')))))))\
                                        )))

In [0]:
df_pnadc = create_age_group(df_pnadc)

In [0]:
def create_job_type(df: DataFrame)-> DataFrame:
  return (df.withColumn("cd_tipo_vinculo",f.when(f.col('VD4009').isin('1','3','5','7'), f.lit(1))\
                        .otherwise(f.when(f.col('V4019')==f.lit(1),f.lit(2))\
                          .otherwise(f.lit(3))))\
                       .withColumn("nm_tipo_vinculo",f.when(f.col('VD4009').isin('1','3','5','7'), f.lit('CLT ou público'))\
                        .otherwise(f.when(f.col('V4019')==f.lit(1),f.lit('CNPJ'))\
                          .otherwise(f.lit('Informal')))))

In [0]:
df_pnadc = create_job_type(df_pnadc)

In [0]:
def calculate_agregated_values_ocupados(df: DataFrame) -> DataFrame:
  return (df.where(f.col('VD4002')==f.lit(1))
  .groupBy("nm_ano","nm_trimestre","cd_uf","cd_faixa_etaria","ds_faixa_etaria",
                    "cd_grau_instrucao_agregado","ds_grau_instrucao_agregado","cd_cnae_dom_div","cd_tipo_vinculo","nm_tipo_vinculo","cd_sexo","ds_sexo").agg(
                      f.sum('vl_horas_trabalhadas').alias('vl_massa_horas_trab'),
                  f.sum(f.col("vl_ocupados")).alias("vl_ocupados"),
                  f.sum(f.col("vl_protecao_social")).alias("vl_protecao_social"),
                 f.sum(f.col("vl_anos_estudos")).alias("vl_anos_estudo_ocupados"),
                 f.sum(f.col("vl_massa_salarial")).alias("vl_massa_salarial")))

In [0]:
df_ocupados = calculate_agregated_values_ocupados(df_pnadc)

In [0]:
def calculate_agregated_values_population(df: DataFrame) -> DataFrame:
  return df.groupBy("nm_ano","nm_trimestre","cd_uf","cd_faixa_etaria","ds_faixa_etaria",
                    "cd_grau_instrucao_agregado","ds_grau_instrucao_agregado","cd_sexo","ds_sexo").agg(
                      f.sum(f.col("V1028")).alias("vl_populacao"),
                   f.sum(f.col("vl_populacao_14anosmais")).alias("vl_populacao_14anosmais"),   
                  f.sum(f.col("vl_desocupados")).alias("vl_desocupados"),
                  f.sum(f.col("vl_subocupados")).alias("vl_subocupados"),
                 f.sum(f.col("vl_subutilizados")).alias("vl_subutilizados"),
                 f.sum(f.col("vl_forca_de_trabalho")).alias("vl_forca_de_trabalho"),
                 f.sum(f.col("vl_forca_de_trabalho_ampliada")).alias("vl_forca_de_trabalho_ampliada"),
                 f.sum('vl_anos_estudos').alias('vl_anos_estudo'))

In [0]:
df_populacao=calculate_agregated_values_population(df_pnadc)

In [0]:
df_ocupados = tcf.add_control_fields(df_ocupados, adf, layer="biz")

var_sink = "{adl_path}{biz}/{path_destination}/{params_to_save_0}".format(adl_path=var_adls_uri, biz=dls['folders']['business'], path_destination=table["path_destination"], params_to_save_0=table["params_to_save_0"])

In [0]:
df_ocupados.write.format('parquet').save(var_sink, header = True, mode='overwrite')

In [0]:
df_populacao = tcf.add_control_fields(df_populacao, adf, layer="biz")

var_sink = "{adl_path}{biz}/{path_destination}/{params_to_save_1}".format(adl_path=var_adls_uri, biz=dls['folders']['business'], path_destination=table["path_destination"], params_to_save_1=table["params_to_save_1"])

In [0]:
df_populacao.write.format('parquet').save(var_sink, header = True, mode='overwrite')